# Data Collection - PART 1
## Notebook to grab links. 
Carmax doesn't give links directly so we need to grab the stockno from each of the banners and then use these to generate links 

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import math
import os
chromedriver = f"{os.environ['HOME']}/.local/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [8]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.carmax.com/search#BT=0&Distance=250&ExposedCategories=249+250+1001+1000+265+999+772&ExposedDimensions=249+250+1001+1000+265+999+772&MaxYear=2019&MinYear=2008&Page=1&PerPage=50&SortKey=0&Zip=98101")
time.sleep(2);

### 1. Read the total number of matches
Get the number of matches so that I know how many time to click the next button 

In [9]:
#matches =
page_selector = '//span[@class="light-blue"]'
matches=driver.find_element_by_xpath(page_selector).text.split()[0].replace(",","")
print(matches)
n = math.ceil(int(matches)/50)
print(n)

1293
26


### 2. Grab from each page multiple banners 
store is a list which stores all the stuff we grab from each page about 50 different entries/banners at a time

In [10]:
store =[]

In [11]:
# 'contains' will find a match on the text, in this case return b tag
page = 1
while(page<=n):
    gross_selector = '//span[@content]'
    for car in driver.find_elements_by_xpath(gross_selector):
        store.append(car.get_attribute('content'))
    print("------")
    #print(store[-1])
    print(len(store))
    
    # find the next button and click it  
    go_button = driver.find_element_by_xpath('//a[@class="pagination--next"]') 
    go_button.click()
    time.sleep(5);
    page+=1


------
150
------
300
------
450
------
600
------
750
------
900
------
1050
------
1200
------
1350
------
1500
------
1650
------
1800
------
1950
------
2100
------
2250
------
2400
------
2550
------
2700
------
2850
------
3000
------
3150
------
3300
------
3450
------
3600
------
3750
------
3900


In [12]:
# looking at what I have got . I get data in this sequence stock_no model details, USD(unwanted), price i 
print(store[:10])
print(len(store))

['16237735 2013 Audi A5 Premium', 'USD', '21998.00', '16237784 2014 Audi A5 Premium Plus', 'USD', '22998.00', '16238238 2013 Audi Q5 Premium Plus', 'USD', '26998.00', '15411470 2013 Audi RS5']
3900


### 3. Iterate through store
Iterate through store and make a order the following
1. a list of list - structured[] - that contains stock_no,brand,model no
2. a list of prices - prices []

In [13]:
row_counter = 0
structured = []
price = []

for data in store:
    if(row_counter%3==0): # pickup the stockno brand model
        main_str = data
        temp = [int(main_str.split(' ',3)[0]),int(main_str.split(' ',3)[1]),main_str.split(' ',3)[2],main_str.split(' ',3)[3]]
        structured.append(temp)
    if(row_counter%3==2): # pick up the price 
        main_str = data
        #print('----',main_str)
        price.append(float(main_str))
    row_counter +=1
print(structured[:10])
print(price[:10])

[[16237735, 2013, 'Audi', 'A5 Premium'], [16237784, 2014, 'Audi', 'A5 Premium Plus'], [16238238, 2013, 'Audi', 'Q5 Premium Plus'], [15411470, 2013, 'Audi', 'RS5'], [15952558, 2015, 'BMW', 'X4 XDrive35i'], [16568229, 2013, 'Cadillac', 'ATS Premium'], [16238577, 2012, 'Cadillac', 'CTS Luxury'], [16171245, 2012, 'Chevrolet', 'Avalanche 1500 LT'], [16171094, 2012, 'Chevrolet', 'Avalanche 1500 LTZ'], [16237838, 2017, 'Chevrolet', 'Camaro LT']]
[21998.0, 22998.0, 26998.0, 41998.0, 34998.0, 21998.0, 16998.0, 21998.0, 27998.0, 25998.0]


In [14]:
print(len(structured))
print(len(price))

1300
1300


### 4. Put everything in a nice panda 

In [15]:
import pandas as pd
headings =['stock_n','year','brand','model']
df_cars=pd.DataFrame(structured,columns=headings)
df_cars.head()

,stock_n,year,brand,model
0,16237735,2013,Audi,A5 Premium
1,16237784,2014,Audi,A5 Premium Plus
2,16238238,2013,Audi,Q5 Premium Plus
3,15411470,2013,Audi,RS5
4,15952558,2015,BMW,X4 XDrive35i


In [16]:
df_cars['price'] = price
df_cars.head()


,stock_n,year,brand,model,price
0,16237735,2013,Audi,A5 Premium,21998.0
1,16237784,2014,Audi,A5 Premium Plus,22998.0
2,16238238,2013,Audi,Q5 Premium Plus,26998.0
3,15411470,2013,Audi,RS5,41998.0
4,15952558,2015,BMW,X4 XDrive35i,34998.0


In [17]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 5 columns):
stock_n    1300 non-null int64
year       1300 non-null int64
brand      1300 non-null object
model      1300 non-null object
price      1300 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 50.9+ KB


### 5. Write to a file car_list.csv

In [21]:
df_cars.to_csv('../data/car_list.csv')

In [19]:
driver.close()

### next -> page_scrap.ipynb